In [1]:
! pip install psycopg2-binary --user

import pandas as pd
import psycopg2
import numpy as np
from getpass import getpass

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# connect to database
connection = psycopg2.connect(
    database  = "postgres",
    user      = "postgres",
    password  = getpass(),
    host      = "movie-rec-scrape.cvslmiksgnix.us-east-1.rds.amazonaws.com",
    port      = '5432'
)


 ················


In [3]:
# cursor object
try:
    c = connection.cursor()
    print("Connected!")
except:
    print("Connection problem chief!")

Connected!


In [4]:
c.execute("SELECT movie_id, username FROM reviews")

In [5]:
result = c.fetchall()
result

[('3120280', 'r-bea623'),
 ('3120280', 'elvira-rizova'),
 ('3120280', 'luciabcn86'),
 ('3120280', 'cmeneses-74957'),
 ('3120280', 'maxbrt-155-905276'),
 ('3120280', 'joleng88'),
 ('3120280', 'ngojhoang'),
 ('3120280', 'MaiWitt'),
 ('3120280', 'ldjg134'),
 ('3120280', 'rynours-18857'),
 ('3120280', 'shannen-l-c'),
 ('3120280', 'mary-179-677383'),
 ('3120280', 'franciscasilva-16742'),
 ('3120280', 'luisbriz'),
 ('3120280', 'serendipity163'),
 ('3120280', 'dostanapat-48310'),
 ('3120280', 'fufu-73839'),
 ('3120280', 'lrczlnsk'),
 ('3120280', 'majnejmizmina'),
 ('3120280', 'almawisly'),
 ('3120280', 'alina-31420'),
 ('3120280', 'row-72826'),
 ('3120280', 'reignchola-36485'),
 ('3120280', 'vinitavkumbhar'),
 ('3120280', 'princess_mary_of_love'),
 ('3120280', 'perlshop'),
 ('3120280', 'susanagascue'),
 ('3120280', 'sukmolelono'),
 ('3120280', 'belllatrixy'),
 ('3120280', 'amandakroghn'),
 ('3120280', 'moosuch'),
 ('3120280', 'sidharth-banerjee25'),
 ('3120280', 'kaz_3307'),
 ('3120280', 'mar

In [6]:
df = pd.DataFrame(result, columns = ['movieid', 'userid'])
df.head()

,movieid,userid
0,3120280,r-bea623
1,3120280,elvira-rizova
2,3120280,luciabcn86
3,3120280,cmeneses-74957
4,3120280,maxbrt-155-905276


In [7]:
url = 'https://datasets.imdbws.com/title.basics.tsv.gz'

In [8]:
df2 = pd.read_csv(url, sep='\t')

/home/ec2-user/anaconda3/envs/python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df2['movieid'] =[i[2:] for i in df2['tconst']]

In [10]:
df2['movieid'] = df2['movieid'].astype(int)

In [11]:
df2.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,movieid
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",2
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",3
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",4
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",5


In [12]:
df = df.merge(df2, how='inner', on='movieid')

MemoryError: 

In [ ]:
df.head()

In [ ]:
df.movieid.value_counts()

In [ ]:
movies = df.movieid.unique().tolist()

In [ ]:
df['users'] = df['userid'].astype('str')

In [ ]:
users = df.userid.unique().tolist()
len(users)

In [ ]:
df.dtypes

In [ ]:
# ! sudo su
! yum update -y
! yum -y install python-pip
! python -V

In [ ]:
! which pip

In [ ]:
# ! python -m pip install tqdm
# # ! python -c 'import tqdm'
# ! python -m pip install gensim
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')



# shuffle usernames
random.shuffle(users)

# extract 90% of users
users_train = [users[i] for i in range(round(0.9*len(users)))]

# split data into train and validation set
train_df = df[df['users'].isin(users_train)]
validation_df = df[~df['users'].isin(users_train)]

train_df.head()

In [ ]:
# list to capture watch history of the users
watched_train = []

# populate the list with the movie ids
for i in tqdm(users_train):
    temp = train_df[train_df["users"] == i]["movieid"].tolist()
    watched_train.append(temp)

In [ ]:
# list to capture purchase history of the customers
watched_val = []

# populate the list with the product codes
for i in tqdm(validation_df['users'].unique()):
    temp = validation_df[validation_df["users"] == i]["movieid"].tolist()
    watched_val.append(temp)

In [ ]:
watched_train = [[str(x) for x in lst] for lst in watched_train]

In [ ]:
watched_train[0]

In [ ]:
watched_val = [[str(x) for x in lst] for lst in watched_val]

In [ ]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(watched_train, progress_per=200)

model.train(watched_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

In [ ]:
# save word2vec model
model.save("word2vec_2.model")

In [ ]:
import gensim

model = gensim.models.Word2Vec.load("word2vec_2.model")

In [ ]:
model.init_sims(replace=True)

In [ ]:
print(model)

In [ ]:
# extract all vectors
X = model[model.wv.vocab]

X.shape

In [ ]:
# ! sudo pip install umap-learn

In [ ]:
train_df['movieid'] = train_df['movieid'].astype(str)
train_df['movieid'][0]

In [ ]:
movies = train_df[["movieid", "primaryTitle"]]

# remove duplicates
movies.drop_duplicates(inplace=True, subset='movieid', keep="last")

# create product-ID and product-description dictionary
movies_dict = movies.groupby('movieid')['primaryTitle'].apply(list).to_dict()

In [ ]:
# test the dictionary
movies_dict['105112']



In [ ]:
def similar_movies(v, n = 6):
    
    # extract most similar movies for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract title and similarity score of the similar movies
    new_ms = []
    for j in ms:
        pair = (movies_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms

In [ ]:
similar_movies(model['105112'])

In [ ]:
def aggregate_vectors(movies):
    movie_vec = []
    for i in movies:
        try:
            movie_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(movie_vec, axis=0)


In [ ]:
len(watched_val[2])

In [ ]:
aggregate_vectors(watched_val[2]).shape

In [ ]:
similar_movies(aggregate_vectors(watched_val[2]))

In [ ]:
similar_movies(aggregate_vectors(watched_val[2][-10:]))